# **Machine Learning**

## Dự đoán đội bóng rổ thắng hay thua trong trận kế tiếp

In [55]:
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


In [56]:
pd.set_option("display.max_rows", 50)

In [57]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [58]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,99,0.454,0.424,0.333,0.232,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,...,0.348,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,26,32,1.500,1.500,1.000,2.000,17.1,44.5,29.8,33.0,7.3,7.4,43.6,40.4,300,98,BRK,125,1,2021,12/22/2020,False
1,240,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,125,0.589,0.538,0.380,0.348,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,...,0.232,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,20,9,1.333,1.333,0.667,2.000,20.3,50.1,29.7,61.1,3.7,12.7,50.0,32.1,267,120,GSW,99,0,2021,12/22/2020,True
2,240,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,116,0.572,0.548,0.430,0.204,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,...,0.383,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,22,14,1.042,1.000,1.000,1.000,16.1,39.6,24.4,47.0,4.3,3.9,100.0,35.9,166,118,LAL,109,1,2021,12/22/2020,True
3,240,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,109,0.576,0.525,0.358,0.383,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,...,0.204,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,33,17,0.874,0.861,0.800,0.800,18.1,38.7,16.9,24.2,4.7,4.5,20.9,40.2,154,114,LAC,116,0,2021,12/22/2020,False
4,240,46,90,0.511,14,35,0.400,15,18,0.833,11,41,52,19,4,6,16,20,121,0.618,0.589,0.389,0.200,28.9,80.4,58.4,41.3,4.0,9.8,14.0,100,121.1,122.1,13,26,0.750,3,8,0.750,6,8,1.0,2,12,14,8,2,2,7,5,...,0.109,19.6,71.1,41.6,47.9,8.0,10.9,5.4,100,122.1,121.1,13,28,1.000,6,13,1.000,4,4,1.0,4,7,8,7,2,2,2,3,33,20,1.025,1.250,0.800,1.333,22.6,26.9,19.3,22.2,3.8,7.0,33.3,39.2,203,126,BOS,122,1,2021,12/23/2020,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,5,33,38,20,3,6,10,14,88,0.498,0.469,0.463,0.238,11.4,68.8,41.3,60.6,3.4,9.0,10.2,100,98.4,107.3,8,18,0.800,3,6,0.500,3,6,1.0,3,9,12,6,1,4,4,3,...,0.176,31.3,88.6,58.7,52.8,7.8,7.0,7.5,100,107.3,98.4,11,20,1.000,3,7,0.500,7,8,1.0,8,10,15,11,2,1,3,4,25,15,1.000,1.000,0.667,2.000,25.1,53.4,25.6,41.9,3.6,3.5,33.3,28.1,203,106,PHI,96,0,2023,4/22/2023,False
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,14,35,49,25,10,8,13,14,112,0.584,0.542,0.262,0.321,36.8,79.5,59.8,61.0,10.6,14.5,11.9,100,118.9,106.1,10,21,0.571,3,6,0.750,10,11,1.0,4,10,13,9,3,3,5,3,...,0.109,20.5,63.2,40.2,47.5,7.4,8.1,11.9,100,106.1,118.9,17,29,0.750,3,8,0.750,4,6,1.0,3,6,9,4,1,1,4,5,37,4,0.820,0.750,1.000,0.500,16.9,31.0,21.6,28.8,2.8,4.0,29.1,35

### Data Cleaning

In [59]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,99,0.454,0.424,0.333,0.232,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,...,0.348,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,26,32,1.500,1.500,1.000,2.000,17.1,44.5,29.8,33.0,7.3,7.4,43.6,40.4,300,98,BRK,125,1,2021,12/22/2020,False
1,240,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,125,0.589,0.538,0.380,0.348,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,...,0.232,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,20,9,1.333,1.333,0.667,2.000,20.3,50.1,29.7,61.1,3.7,12.7,50.0,32.1,267,120,GSW,99,0,2021,12/22/2020,True
2,240,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,116,0.572,0.548,0.430,0.204,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,...,0.383,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,22,14,1.042,1.000,1.000,1.000,16.1,39.6,24.4,47.0,4.3,3.9,100.0,35.9,166,118,LAL,109,1,2021,12/22/2020,True
3,240,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,109,0.576,0.525,0.358,0.383,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,...,0.204,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,33,17,0.874,0.861,0.800,0.800,18.1,38.7,16.9,24.2,4.7,4.5,20.9,40.2,154,114,LAC,116,0,2021,12/22/2020,False
4,240,46,90,0.511,14,35,0.400,15,18,0.833,11,41,52,19,4,6,16,20,121,0.618,0.589,0.389,0.200,28.9,80.4,58.4,41.3,4.0,9.8,14.0,100,121.1,122.1,13,26,0.750,3,8,0.750,6,8,1.0,2,12,14,8,2,2,7,5,...,0.109,19.6,71.1,41.6,47.9,8.0,10.9,5.4,100,122.1,121.1,13,28,1.000,6,13,1.000,4,4,1.0,4,7,8,7,2,2,2,3,33,20,1.025,1.250,0.800,1.333,22.6,26.9,19.3,22.2,3.8,7.0,33.3,39.2,203,126,BOS,122,1,2021,12/23/2020,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,5,33,38,20,3,6,10,14,88,0.498,0.469,0.463,0.238,11.4,68.8,41.3,60.6,3.4,9.0,10.2,100,98.4,107.3,8,18,0.800,3,6,0.500,3,6,1.0,3,9,12,6,1,4,4,3,...,0.176,31.3,88.6,58.7,52.8,7.8,7.0,7.5,100,107.3,98.4,11,20,1.000,3,7,0.500,7,8,1.0,8,10,15,11,2,1,3,4,25,15,1.000,1.000,0.667,2.000,25.1,53.4,25.6,41.9,3.6,3.5,33.3,28.1,203,106,PHI,96,0,2023,4/22/2023,False
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,14,35,49,25,10,8,13,14,112,0.584,0.542,0.262,0.321,36.8,79.5,59.8,61.0,10.6,14.5,11.9,100,118.9,106.1,10,21,0.571,3,6,0.750,10,11,1.0,4,10,13,9,3,3,5,3,...,0.109,20.5,63.2,40.2,47.5,7.4,8.1,11.9,100,106.1,118.9,17,29,0.750,3,8,0.750,4,6,1.0,3,6,9,4,1,1,4,5,37,4,0.820,0.750,1.000,0.500,16.9,31.0,21.6,28.8,2.8,4.0,29.1,35

Để dự đoán kết quả của trận đấu tiếp theo, chúng ta có thể tạo một thuộc tính tên là 'target' cho từng đội bằng cách đẩy dữ liệu của cột 'won' xuống một hàng

In [60]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group



df = df.groupby("team", group_keys=False).apply(add_target)

In [61]:
df.tail(50)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
5140,240,38,98,0.388,5,29,0.172,18,22,0.818,14,31,45,23,13,4,11,17,99,0.460,0.413,0.296,0.224,23.3,72.1,43.7,60.5,13.0,7.3,9.3,100,98.9,111.8,10,25,0.750,2,7,0.286,5,7,1.000,5,8,8,8,3,2,5,4,...,27.9,76.7,56.3,59.5,6.0,10.1,14.3,100,111.8,98.9,12,23,1.000,4,7,0.571,6,7,1.000,4,11,12,7,3,2,6,3,29,16,1.000,1.000,1.000,0.667,12.1,22.2,17.2,28.9,4.5,4.3,33.3,35.1,148,105,BOS,112,1,2023,4/15/2023,False,False
5141,240,42,88,0.477,13,33,0.394,15,18,0.833,12,46,58,25,6,7,16,15,112,0.584,0.551,0.375,0.205,27.9,76.7,56.3,59.5,6.0,10.1,14.3,100,111.8,98.9,12,23,1.000,4,7,0.571,6,7,1.000,4,11,12,7,3,2,6,3,...,23.3,72.1,43.7,60.5,13.0,7.3,9.3,100,98.9,111.8,10,25,0.750,2,7,0.286,5,7,1.000,5,8,8,8,3,2,5,4,24,3,0.761,0.750,0.700,1.000,20.9,33.7,20.9,35.5,4.0,6.3,19.2,33.2,145,124,ATL,99,0,2023,4/15/2023,True,True
5142,240,37,88,0.420,8,29,0.276,19,22,0.864,17,34,51,18,11,1,13,20,101,0.517,0.466,0.330,0.250,38.6,75.6,57.3,48.6,12.1,1.9,11.7,100,111.1,106.7,11,24,0.750,3,10,0.500,4,4,1.000,5,8,10,6,4,1,5,4,...,24.4,61.4,42.7,55.6,9.9,3.4,13.2,100,106.7,111.1,14,30,0.750,6,16,0.667,4,6,1.000,5,11,14,8,4,1,5,3,38,9,0.922,1.000,1.000,2.000,14.1,39.5,19.6,42.7,9.6,1.9,66.7,35.9,157,121,CLE,97,1,2023,4/15/2023,True,False
5143,240,36,83,0.434,10,31,0.323,15,21,0.714,11,27,38,20,9,2,14,17,97,0.526,0.494,0.373,0.253,24.4,61.4,42.7,55.6,9.9,3.4,13.2,100,106.7,111.1,14,30,0.750,6,16,0.667,4,6,1.000,5,11,14,8,4,1,5,3,...,38.6,75.6,57.3,48.6,12.1,1.9,11.7,100,111.1,106.7,11,24,0.750,3,10,0.500,4,4,1.000,5,8,10,6,4,1,5,4,27,19,0.922,0.875,1.000,1.333,20.9,25.4,16.5,27.3,6.8,3.4,30.7,39.3,191,113,NYK,101,0,2023,4/15/2023,False,True
5144,240,39,70,0.557,13,29,0.448,10,15,0.667,5,30,35,23,3,5,19,15,101,0.659,0.650,0.414,0.214,17.2,68.2,47.9,59.0,3.4,10.9,19.9,100,114.3,137.0,12,18,0.800,4,6,1.000,4,5,1.000,2,9,10,7,2,3,4,3,...,31.8,82.8,52.1,76.2,15.8,7.3,7.7,100,137.0,114.3,9,21,1.000,7,13,1.000,11,11,1.000,5,5,7,13,5,2,4,3,26,20,1.250,1.250,1.000,0.733,21.8,65.3,26.0,55.1,16.7,7.1,16.0,32.0,250,124,PHI,121,1,2023,4/15/2023,False,False
5145,240,42,89,0.472,21,43,0.488,16,16,1.000,14,24,38,32,14,3,8,17,121,0.630,0.590,0.483,0.180,31.8,82.8,52.1,76.2,15.8,7.3,7.7,100,137.0,114.3,9,21,1.000,7,13,1.000,11,11,1.000,5,5,7,13,5,2,4,3,...,17.2,68.2,47.9,59.0,3.4,10.9,19.9,100,114.3,137.0,12,18,0.800,4,6,1.000,4,5,1.000,2,9,10,7,2,3,4,3,30,-2,1.000,1.000,1.000,1.000,19.0,32.5,22.7,33.0,3.0,10.4,40.0,42.6,233,146,BRK,101,0,2023,4/15/2023,True,True
5146,240,43,90,0.478,16,50,0.320,21,27,0.778,9,32,41,31,7,7,15,25,123,0.604,0.567,0.556,0.300,21.4,65.3,45.1,72.1,6.7,10.6,12.8,100,117.8,120.7,11,20,1.000,6,14,0.500,7,8,1.000,2,7,9,11,2,4,5,5,...,34.7,78.6,54.9,40.9,9.6,7.5,10.4,100,120.7,117.8,13,27,1.000,4,8,0.667,14,14,1.000,5,11,16,5,3,1,4,4,38,10,1.000,1.000,0.750,1.077,15.2,44.2,28.6,21.1,4.0,9.3,50.0,33.5,206,125,SAC,126,1,2023,4/15/2023,False,False
5147,240,44,98,0.449,12,32,0.375,26,32,0.813,17,33,50,18,10,3,13,23,126,0.562,0.510,0.327,0.327,34.7,78.6,54.9,40.9,9.6,7.5,10.4,100,120.7,117.8,13,27,1.000,4,8,0.667,14,14,1.000,5,11,16,5,3,1,4,4,...,21.4,65.3,45.1,72.1,6.7,10.6,12.8,100,117.8,120.7,11,20,1.000,6,14,0.500,7,8,1.000,2,7,9,11,2,4,5,5,30,11,0.862,1.000

Ở đội có mã là PHI, ta thấy 'target' ở thời gian gần nhất là null bởi vì ta chưa biết được kết quả tiếp theo của đội đó

In [62]:
df[df["team"] == "PHI"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
21,240,41,87,0.471,8,28,0.286,23,30,0.767,10,37,47,22,11,8,16,25,113,0.564,0.517,0.322,0.345,22.2,88.1,54.0,53.7,10.4,13.8,13.8,100,106.9,101.2,10,17,1.000,3,6,1.000,8,9,1.0,4,12,14,7,3,3,4,4,...,11.9,77.8,46.0,71.8,6.6,6.8,16.6,100,101.2,106.9,11,22,1.000,4,8,1.000,6,7,1.000,2,9,11,15,4,1,6,5,31,14,1.215,1.500,1.000,3.000,6.9,36.1,18.6,73.2,5.4,4.9,40.0,36.6,207,115,WAS,107,0,2021,12/23/2020,True,True
46,240,41,88,0.466,11,31,0.355,16,22,0.727,12,35,47,20,9,6,12,19,109,0.558,0.528,0.352,0.250,28.6,72.9,52.2,48.8,9.6,11.1,10.9,100,116.0,94.8,10,20,0.750,3,6,0.750,5,8,1.0,4,8,10,6,4,2,3,4,...,27.1,71.4,47.8,59.4,6.4,7.0,14.7,100,94.8,116.0,11,15,0.733,4,6,1.000,6,8,1.000,5,5,9,4,2,2,5,4,25,1,0.787,0.800,1.000,0.533,20.0,21.5,19.2,24.9,3.3,6.7,24.2,35.9,139,125,NYK,89,1,2021,12/26/2020,True,False
62,240,33,79,0.418,12,37,0.324,16,22,0.727,8,31,39,24,9,5,21,19,94,0.530,0.494,0.468,0.278,19.0,73.8,46.4,72.7,9.0,7.4,19.1,100,93.6,117.5,5,10,1.000,4,7,0.667,5,7,1.0,4,9,9,5,3,2,6,4,...,26.2,81.0,53.6,64.6,12.9,14.3,14.0,100,117.5,93.6,11,18,1.000,3,4,0.750,5,6,1.000,6,8,14,7,3,2,4,5,24,29,1.083,1.083,0.750,2.000,26.7,49.0,31.2,31.9,5.6,13.9,50.0,34.0,200,106,CLE,118,1,2021,12/27/2020,False,True
101,240,32,84,0.381,8,31,0.258,28,33,0.848,9,45,54,21,10,10,18,15,100,0.508,0.429,0.369,0.393,19.6,81.8,53.5,65.6,9.6,22.2,15.4,100,96.3,89.6,11,20,0.550,3,6,0.500,14,16,1.0,4,13,16,7,2,2,5,4,...,18.2,80.4,46.5,65.6,11.6,13.2,16.5,100,89.6,96.3,8,23,1.000,4,9,0.571,9,9,1.000,2,6,8,9,5,2,4,6,24,12,0.842,1.000,0.667,2.000,8.0,28.8,17.5,47.8,6.4,5.1,68.0,30.0,152,108,TOR,93,0,2021,12/29/2020,True,True
126,240,44,91,0.484,15,33,0.455,13,15,0.867,7,47,54,23,7,10,14,23,116,0.594,0.566,0.363,0.165,15.6,82.5,52.9,52.3,6.8,14.9,12.5,100,112.7,89.3,8,15,0.750,5,7,1.000,6,8,1.0,3,9,10,8,2,2,3,4,...,17.5,84.4,47.1,51.5,1.9,8.6,9.5,100,89.3,112.7,8,18,0.533,3,6,0.500,5,5,1.000,4,8,10,4,1,2,2,3,19,10,0.750,0.750,1.000,0.625,13.9,35.9,19.8,45.8,1.9,8.0,20.2,30.5,131,117,ORL,92,1,2021,12/31/2020,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,240,53,101,0.525,14,32,0.438,14,19,0.737,13,37,50,31,10,5,13,23,134,0.613,0.594,0.317,0.188,28.9,78.7,54.3,58.5,9.6,9.8,10.6,100,128.8,100.9,8,17,0.750,4,9,1.000,5,7,1.0,4,7,10,9,2,5,3,5,...,21.3,71.1,45.7,62.9,5.8,8.7,15.9,100,100.9,128.8,16,29,0.552,6,9,0.750,8,8,1.000,3,7,10,7,3,2,4,5,46,0,0.707,0.655,0.818,0.636,12.2,29.8,20.8,35.7,3.1,5.5,23.7,36.2,123,131,BRK,105,1,2023,4/9/2023,True,True
5145,240,42,89,0.472,21,43,0.488,16,16,1.000,14,24,38,32,14,3,8,17,121,0.630,0.590,0.483,0.180,31.8,82.8,52.1,76.2,15.8,7.3,7.7,100,137.0,114.3,9,21,1.000,7,13,1.000,11,11,1.0,5,5,7,13,5,2,4,3,...,17.2,68.2,47.9,59.0,3.4,10.9,19.9,100,114.3,137.0,12,18,0.800,4,6,1.000,4,5,1.000,2,9,10,7,2,3,4,3,30,-2,1.000,1.000,1.000,1.000,19.0,32.5,22.7,33.0,3.0,10.4,40.0,

In [63]:
df["target"][pd.isnull(df["target"])] = 2 # với target là null, ta sẽ cho tương đương với giá trị là 2
df["target"] = df["target"].astype(int, errors="ignore") # target là false thì tương đương với giá trị là 0, true là 1

C:\Users\huyda\AppData\Local\Temp\ipykernel_16316\1188258689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2 # với target là null, ta sẽ cho tương đương với giá trị là 2


In [64]:
df[df["team"] == "PHI"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
21,240,41,87,0.471,8,28,0.286,23,30,0.767,10,37,47,22,11,8,16,25,113,0.564,0.517,0.322,0.345,22.2,88.1,54.0,53.7,10.4,13.8,13.8,100,106.9,101.2,10,17,1.000,3,6,1.000,8,9,1.0,4,12,14,7,3,3,4,4,...,11.9,77.8,46.0,71.8,6.6,6.8,16.6,100,101.2,106.9,11,22,1.000,4,8,1.000,6,7,1.000,2,9,11,15,4,1,6,5,31,14,1.215,1.500,1.000,3.000,6.9,36.1,18.6,73.2,5.4,4.9,40.0,36.6,207,115,WAS,107,0,2021,12/23/2020,True,1
46,240,41,88,0.466,11,31,0.355,16,22,0.727,12,35,47,20,9,6,12,19,109,0.558,0.528,0.352,0.250,28.6,72.9,52.2,48.8,9.6,11.1,10.9,100,116.0,94.8,10,20,0.750,3,6,0.750,5,8,1.0,4,8,10,6,4,2,3,4,...,27.1,71.4,47.8,59.4,6.4,7.0,14.7,100,94.8,116.0,11,15,0.733,4,6,1.000,6,8,1.000,5,5,9,4,2,2,5,4,25,1,0.787,0.800,1.000,0.533,20.0,21.5,19.2,24.9,3.3,6.7,24.2,35.9,139,125,NYK,89,1,2021,12/26/2020,True,0
62,240,33,79,0.418,12,37,0.324,16,22,0.727,8,31,39,24,9,5,21,19,94,0.530,0.494,0.468,0.278,19.0,73.8,46.4,72.7,9.0,7.4,19.1,100,93.6,117.5,5,10,1.000,4,7,0.667,5,7,1.0,4,9,9,5,3,2,6,4,...,26.2,81.0,53.6,64.6,12.9,14.3,14.0,100,117.5,93.6,11,18,1.000,3,4,0.750,5,6,1.000,6,8,14,7,3,2,4,5,24,29,1.083,1.083,0.750,2.000,26.7,49.0,31.2,31.9,5.6,13.9,50.0,34.0,200,106,CLE,118,1,2021,12/27/2020,False,1
101,240,32,84,0.381,8,31,0.258,28,33,0.848,9,45,54,21,10,10,18,15,100,0.508,0.429,0.369,0.393,19.6,81.8,53.5,65.6,9.6,22.2,15.4,100,96.3,89.6,11,20,0.550,3,6,0.500,14,16,1.0,4,13,16,7,2,2,5,4,...,18.2,80.4,46.5,65.6,11.6,13.2,16.5,100,89.6,96.3,8,23,1.000,4,9,0.571,9,9,1.000,2,6,8,9,5,2,4,6,24,12,0.842,1.000,0.667,2.000,8.0,28.8,17.5,47.8,6.4,5.1,68.0,30.0,152,108,TOR,93,0,2021,12/29/2020,True,1
126,240,44,91,0.484,15,33,0.455,13,15,0.867,7,47,54,23,7,10,14,23,116,0.594,0.566,0.363,0.165,15.6,82.5,52.9,52.3,6.8,14.9,12.5,100,112.7,89.3,8,15,0.750,5,7,1.000,6,8,1.0,3,9,10,8,2,2,3,4,...,17.5,84.4,47.1,51.5,1.9,8.6,9.5,100,89.3,112.7,8,18,0.533,3,6,0.500,5,5,1.000,4,8,10,4,1,2,2,3,19,10,0.750,0.750,1.000,0.625,13.9,35.9,19.8,45.8,1.9,8.0,20.2,30.5,131,117,ORL,92,1,2021,12/31/2020,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,240,53,101,0.525,14,32,0.438,14,19,0.737,13,37,50,31,10,5,13,23,134,0.613,0.594,0.317,0.188,28.9,78.7,54.3,58.5,9.6,9.8,10.6,100,128.8,100.9,8,17,0.750,4,9,1.000,5,7,1.0,4,7,10,9,2,5,3,5,...,21.3,71.1,45.7,62.9,5.8,8.7,15.9,100,100.9,128.8,16,29,0.552,6,9,0.750,8,8,1.000,3,7,10,7,3,2,4,5,46,0,0.707,0.655,0.818,0.636,12.2,29.8,20.8,35.7,3.1,5.5,23.7,36.2,123,131,BRK,105,1,2023,4/9/2023,True,1
5145,240,42,89,0.472,21,43,0.488,16,16,1.000,14,24,38,32,14,3,8,17,121,0.630,0.590,0.483,0.180,31.8,82.8,52.1,76.2,15.8,7.3,7.7,100,137.0,114.3,9,21,1.000,7,13,1.000,11,11,1.0,5,5,7,13,5,2,4,3,...,17.2,68.2,47.9,59.0,3.4,10.9,19.9,100,114.3,137.0,12,18,0.800,4,6,1.000,4,5,1.000,2,9,10,7,2,3,4,3,30,-2,1.000,1.000,1.000,1.000,19.0,32.5,22.7,33.0,3.0,10.4,40.0,42.6,233,146,BRK,10

In [65]:
df["won"].value_counts()

False    2595
True     2595
Name: won, dtype: int64

In [66]:
df["target"].value_counts()

0    2580
1    2580
2      30
Name: target, dtype: int64

Loại bỏ các cột không có giá trị

In [67]:
nulls = pd.isnull(df).sum()

In [68]:
nulls = nulls[nulls > 0]

In [69]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [70]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [71]:
df = df[valid_columns].copy()

In [72]:
df


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,99,0.454,0.424,0.333,0.232,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,...,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,26,32,1.500,1.500,1.000,2.000,17.1,44.5,29.8,33.0,7.3,7.4,43.6,40.4,300,98,BRK,125,1,2021,12/22/2020,False,0
1,240,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,125,0.589,0.538,0.380,0.348,27.7,77.2,54.8,57.1,10.0,10.6,15.9,100,113.1,89.6,10,17,1.000,4,7,1.000,7,7,1.0,3,9,11,5,3,2,3,3,...,22.8,72.3,45.2,70.3,5.4,10.5,14.2,100,89.6,113.1,7,21,1.000,2,10,1.000,4,8,1.0,4,6,8,10,2,2,4,4,20,9,1.333,1.333,0.667,2.000,20.3,50.1,29.7,61.1,3.7,12.7,50.0,32.1,267,120,GSW,99,0,2021,12/22/2020,True,1
2,240,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,116,0.572,0.548,0.430,0.204,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,...,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,22,14,1.042,1.000,1.000,1.000,16.1,39.6,24.4,47.0,4.3,3.9,100.0,35.9,166,118,LAL,109,1,2021,12/22/2020,True,1
3,240,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,109,0.576,0.525,0.358,0.383,21.6,77.1,52.9,57.9,3.9,3.8,16.7,100,105.3,112.1,8,17,1.000,3,8,0.500,5,7,1.0,4,11,12,8,2,1,4,5,...,22.9,78.4,47.1,50.0,9.7,5.8,13.6,100,112.1,105.3,13,26,0.800,5,8,0.667,5,5,1.0,4,6,6,6,2,1,5,5,33,17,0.874,0.861,0.800,0.800,18.1,38.7,16.9,24.2,4.7,4.5,20.9,40.2,154,114,LAC,116,0,2021,12/22/2020,False,1
4,240,46,90,0.511,14,35,0.400,15,18,0.833,11,41,52,19,4,6,16,20,121,0.618,0.589,0.389,0.200,28.9,80.4,58.4,41.3,4.0,9.8,14.0,100,121.1,122.1,13,26,0.750,3,8,0.750,6,8,1.0,2,12,14,8,2,2,7,5,...,19.6,71.1,41.6,47.9,8.0,10.9,5.4,100,122.1,121.1,13,28,1.000,6,13,1.000,4,4,1.0,4,7,8,7,2,2,2,3,33,20,1.025,1.250,0.800,1.333,22.6,26.9,19.3,22.2,3.8,7.0,33.3,39.2,203,126,BOS,122,1,2021,12/23/2020,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,240,33,80,0.413,9,37,0.243,13,19,0.684,5,33,38,20,3,6,10,14,88,0.498,0.469,0.463,0.238,11.4,68.8,41.3,60.6,3.4,9.0,10.2,100,98.4,107.3,8,18,0.800,3,6,0.500,3,6,1.0,3,9,12,6,1,4,4,3,...,31.3,88.6,58.7,52.8,7.8,7.0,7.5,100,107.3,98.4,11,20,1.000,3,7,0.500,7,8,1.0,8,10,15,11,2,1,3,4,25,15,1.000,1.000,0.667,2.000,25.1,53.4,25.6,41.9,3.6,3.5,33.3,28.1,203,106,PHI,96,0,2023,4/22/2023,False,2
5186,240,41,84,0.488,9,22,0.409,21,27,0.778,14,35,49,25,10,8,13,14,112,0.584,0.542,0.262,0.321,36.8,79.5,59.8,61.0,10.6,14.5,11.9,100,118.9,106.1,10,21,0.571,3,6,0.750,10,11,1.0,4,10,13,9,3,3,5,3,...,20.5,63.2,40.2,47.5,7.4,8.1,11.9,100,106.1,118.9,17,29,0.750,3,8,0.750,4,6,1.0,3,6,9,4,1,1,4,5,37,4,0.820,0.750,1.000,0.500,16.9,31.0,21.6,28.8,2.8,4.0,29.1,35.8,140,128,LAC,100,1,2023,4/22/

### Machine Learning

Chúng sẽ không muốn sử dụng hết các cột để luyện model vì một số lý do như:
<br>    - Các cột có thể tương quan với nhau nên một số thuật toán không thể xử lý tốt
<br>    - Một số cột có thể gây ra overfitting hay nhiều vấn đề khác cho model
<br> Nên trước khi luyện một model, chúng ta có thể sử dụng FeatureSelector để chọn một set các cột nhất định để luyện model

In [73]:
rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3) # chia data thành các bộ nhỏ hơn để học máy (bộ dùng để train và bộ dùng để test)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

RidgeClassifier hoạt động tốt hơn nếu như chúng ta scale dữ liệu sao cho các cột nằm trải dài từ giá trị 0 đến 1

In [74]:

removed_columns = ["season", "date", "won", "target", "team", "team_opp"] # những cột không nên scale
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [75]:
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [76]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.350,0.611111,0.236585,0.296296,0.433962,0.372323,0.307692,0.422222,0.497110,0.48,0.450,0.500000,0.428571,0.272727,0.315789,0.586207,0.629630,0.305882,0.193622,0.248988,0.388889,0.320132,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,...,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,0.294118,0.694444,1.000,1.000,1.0000,0.137001,0.146241,0.330519,0.227723,0.202381,0.073,0.074,0.408805,0.234917,1.000000,0.168831,BRK,0.611765,1.0,2021,12/22/2020,False,0
1,0.0,0.475,0.481481,0.439024,0.481481,0.471698,0.579901,0.589744,0.622222,0.729769,0.48,0.700,0.717391,0.380952,0.500000,0.368421,0.655172,0.555556,0.611765,0.501139,0.479757,0.470486,0.511551,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,...,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,0.176471,0.375000,0.833,0.833,0.4672,0.137001,0.179197,0.398070,0.226623,0.536905,0.037,0.127,0.475891,0.128370,0.829897,0.454545,GSW,0.305882,0.0,2021,12/22/2020,True,1
2,0.0,0.525,0.500000,0.478049,0.444444,0.566038,0.449753,0.282051,0.333333,0.619942,0.40,0.325,0.347826,0.333333,0.454545,0.157895,0.517241,0.814815,0.505882,0.462415,0.500000,0.557292,0.273927,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,...,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,0.215686,0.444444,0.542,0.500,1.0000,0.065085,0.135942,0.271411,0.168317,0.369048,0.043,0.039,1.000000,0.177150,0.309278,0.428571,LAL,0.423529,1.0,2021,12/22/2020,True,1
3,0.0,0.375,0.277778,0.468293,0.259259,0.358491,0.383855,0.538462,0.600000,0.673410,0.28,0.525,0.456522,0.333333,0.181818,0.105263,0.620690,0.481481,0.423529,0.471526,0.453441,0.432292,0.569307,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,...,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,0.431373,0.486111,0.374,0.361,0.6800,0.050701,0.156540,0.260555,0.085809,0.097619,0.047,0.045,0.170860,0.232349,0.247423,0.376623,LAC,0.505882,0.0,2021,12/22/2020,False,1
4

Chọn 30 thuộc tính tốt nhất để học máy

In [77]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [78]:
predictors = list(selected_columns[sfs.get_support()])

In [79]:
predictors

['fg%',
 '3p',
 '3pa',
 '3p%',
 'pts',
 'efg%',
 'ast%',
 'usg%',
 'ortg',
 'stl_max',
 'blk_max',
 '+/-_max',
 'ts%_max',
 'efg%_max',
 'ftr_max',
 'stl%_max',
 'ortg_max',
 'drtg_max',
 'total',
 'fg_opp',
 'tov_opp',
 'blk%_opp',
 'usg%_opp',
 'drtg_opp',
 'ft%_max_opp',
 'orb_max_opp',
 '+/-_max_opp',
 '3par_max_opp',
 'orb%_max_opp',
 'tov%_max_opp']

Chia dữ liệu theo mùa và dùng dữ liệu từ các mùa cũ để dự đoán mùa mới
<br> Ví dụ nếu có các mùa từ 2016-2023 thì chúng ta sẽ dùng dữ liệu mùa 2016-2017 đề dự đoán cho mùa 2018. Sau đó sẽ dùng 2016, 2017, 2018 để dự đoán cho 2019 và cứ tiếp tục như thế cho đến mùa 2023

In [80]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors]) # dự đoán bộ test
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [81]:
predictions = backtest(df, rr, predictors)

In [82]:
predictions

,actual,prediction
4988,0,1
4989,1,1
4990,1,0
4991,0,1
4992,1,0
...,...,...
5185,2,1
5186,2,1
5187,2,0
5188,2,0


In [83]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.4158415841584158

#### Cải thiện model

Nếu để ý thì trong thể thao, các trận chơi ở sân nhà thì có cơ hội thắng cao hơn chơi ở sân khách

In [84]:

df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0]) # lấy số lượng trận thắng/thua chia cho số lượng tổng

home
0.0    0.452023
1.0    0.547977
dtype: float64

In [85]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.350,0.611111,0.236585,0.296296,0.433962,0.372323,0.307692,0.422222,0.497110,0.48,0.450,0.500000,0.428571,0.272727,0.315789,0.586207,0.629630,0.305882,0.193622,0.248988,0.388889,0.320132,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,...,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,0.294118,0.694444,1.000,1.000,1.0000,0.137001,0.146241,0.330519,0.227723,0.202381,0.073,0.074,0.408805,0.234917,1.000000,0.168831,BRK,0.611765,1.0,2021,12/22/2020,False,0
1,0.0,0.475,0.481481,0.439024,0.481481,0.471698,0.579901,0.589744,0.622222,0.729769,0.48,0.700,0.717391,0.380952,0.500000,0.368421,0.655172,0.555556,0.611765,0.501139,0.479757,0.470486,0.511551,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,...,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,0.176471,0.375000,0.833,0.833,0.4672,0.137001,0.179197,0.398070,0.226623,0.536905,0.037,0.127,0.475891,0.128370,0.829897,0.454545,GSW,0.305882,0.0,2021,12/22/2020,True,1
2,0.0,0.525,0.500000,0.478049,0.444444,0.566038,0.449753,0.282051,0.333333,0.619942,0.40,0.325,0.347826,0.333333,0.454545,0.157895,0.517241,0.814815,0.505882,0.462415,0.500000,0.557292,0.273927,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,...,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,0.215686,0.444444,0.542,0.500,1.0000,0.065085,0.135942,0.271411,0.168317,0.369048,0.043,0.039,1.000000,0.177150,0.309278,0.428571,LAL,0.423529,1.0,2021,12/22/2020,True,1
3,0.0,0.375,0.277778,0.468293,0.259259,0.358491,0.383855,0.538462,0.600000,0.673410,0.28,0.525,0.456522,0.333333,0.181818,0.105263,0.620690,0.481481,0.423529,0.471526,0.453441,0.432292,0.569307,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,...,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,0.431373,0.486111,0.374,0.361,0.6800,0.050701,0.156540,0.260555,0.085809,0.097619,0.047,0.045,0.170860,0.232349,0.247423,0.376623,LAC,0.505882,0.0,2021,12/22/2020,False,1
4

Để ý với dataframe hiện tại của chúng ta, khi chúng ta đưa ra dự đoán, chúng ta chỉ nhìn trận đấu hiện tại để dự đoán kết quả tiếp theo. Tuy nhiên trong thực tế, ở một trận đấu cụ thể, đội chơi đó có thể chơi tốt hơn hoặc tệ hơn so với cách chơi thông thường của đội bóng đó. Vì vậy, chúng ta nên sử dụng trung bình các trận đấu gần đây của một đội để dự đoán kết quả tiếp theo

In [86]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [87]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.350,0.611111,0.236585,0.296296,0.433962,0.372323,0.307692,0.422222,0.497110,0.48,0.450,0.500000,0.428571,0.272727,0.315789,0.586207,0.629630,0.305882,0.193622,0.248988,0.388889,0.320132,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,...,0.470486,0.511551,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,0.294118,0.694444,1.000,1.000,1.0000,0.137001,0.146241,0.330519,0.227723,0.202381,0.073,0.074,0.408805,0.234917,1.000000,0.168831,0.611765,1.0,False,GSW,2021
1,0.0,0.475,0.481481,0.439024,0.481481,0.471698,0.579901,0.589744,0.622222,0.729769,0.48,0.700,0.717391,0.380952,0.500000,0.368421,0.655172,0.555556,0.611765,0.501139,0.479757,0.470486,0.511551,0.547974,0.556503,0.633333,0.4816,0.467290,0.363014,0.626050,0.0,0.449344,0.169249,0.368421,0.259259,1.000000,0.272727,0.210526,1.000000,0.272727,0.20,1.0,0.153846,0.333333,0.269231,0.136364,0.3,0.2,0.222222,0.25,...,0.388889,0.320132,0.443497,0.452026,0.366667,0.6928,0.252336,0.359589,0.554622,0.0,0.169249,0.449344,0.210526,0.407407,1.000000,0.090909,0.368421,1.000000,0.136364,0.24,1.0,0.230769,0.166667,0.153846,0.363636,0.2,0.2,0.333333,0.50,0.176471,0.375000,0.833,0.833,0.4672,0.137001,0.179197,0.398070,0.226623,0.536905,0.037,0.127,0.475891,0.128370,0.829897,0.454545,0.305882,0.0,True,BRK,2021
2,0.0,0.525,0.500000,0.478049,0.444444,0.566038,0.449753,0.282051,0.333333,0.619942,0.40,0.325,0.347826,0.333333,0.454545,0.157895,0.517241,0.814815,0.505882,0.462415,0.500000,0.557292,0.273927,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,...,0.432292,0.569307,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,0.215686,0.444444,0.542,0.500,1.0000,0.065085,0.135942,0.271411,0.168317,0.369048,0.043,0.039,1.000000,0.177150,0.309278,0.428571,0.423529,1.0,True,LAC,2021
3,0.0,0.375,0.277778,0.468293,0.259259,0.358491,0.383855,0.538462,0.600000,0.673410,0.28,0.525,0.456522,0.333333,0.181818,0.105263,0.620690,0.481481,0.423529,0.471526,0.453441,0.432292,0.569307,0.417910,0.554371,0.580556,0.4944,0.182243,0.130137,0.659664,0.0,0.356377,0.437426,0.263158,0.259259,1.000000,0.181818,0.263158,0.333333,0.181818,0.20,1.0,0.230769,0.444444,0.307692,0.272727,0.2,0.1,0.333333,0.75,...,0.557292,0.273927,0.445629,0.582090,0.419444,0.3680,0.453271,0.198630,0.529412,0.0,0.437426,0.356377,0.526316,0.592593,0.640288,0.363636,0.263158,0.556000,0.181818,0.12,1.0,0.230769,0.166667,0.076923,0.181818,0.2,0.1,0.444444,0.75,0.431373,0.486111,0.374,0.361,0.6800,0.050701,0.156540,0.260555,0.085809,0.097619,0.047,0.045,0.170860,0.232349,0.247423,0.376623,0.505882

In [88]:
def find_team_averages(team):
    rolling = team.rolling(5).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\huyda\AppData\Local\Temp\ipykernel_16316\4054791798.py:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['team'], dtype='object')
  rolling = team.rolling(5).mean()


In [89]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [90]:

df_rolling


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,0.0,0.280,0.229630,0.396098,0.348148,0.452830,0.436244,0.312821,0.360000,0.619364,0.224,0.345,0.269565,0.338095,0.245455,0.221053,0.406897,0.362963,0.258824,0.405467,0.428340,0.563542,0.359736,0.293817,0.388913,0.302778,0.61504,0.274766,0.251370,0.492437,0.0,0.335399,0.522527,0.431579,0.444444,0.503597,0.309091,0.252632,0.540267,0.227273,0.224,0.9112,0.123077,0.233333,0.215385,0.218182,0.20,0.22,0.288889,0.45,...,0.489879,0.463542,0.235314,0.611514,0.706183,0.697222,0.47456,0.446729,0.332877,0.441176,0.0,0.522527,0.335399,0.378947,0.355556,0.685252,0.363636,0.368421,0.623200,0.272727,0.232,1.0000,0.323077,0.377778,0.315385,0.309091,0.32,0.26,0.400000,0.55,0.290196,0.541667,0.4128,0.4126,0.59968,0.062596,0.267559,0.354162,

Kết hợp dataframe trung bình với dataframe của chúng ta

In [91]:
rolling_cols = [f"{col}_5" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [92]:
df = df.dropna()

In [93]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,efg%_opp_5,3par_opp_5,ftr_opp_5,orb%_opp_5,drb%_opp_5,trb%_opp_5,ast%_opp_5,stl%_opp_5,blk%_opp_5,tov%_opp_5,usg%_opp_5,ortg_opp_5,drtg_opp_5,fg_max_opp_5,fga_max_opp_5,fg%_max_opp_5,3p_max_opp_5,3pa_max_opp_5,3p%_max_opp_5,ft_max_opp_5,fta_max_opp_5,ft%_max_opp_5,orb_max_opp_5,drb_max_opp_5,trb_max_opp_5,ast_max_opp_5,stl_max_opp_5,blk_max_opp_5,tov_max_opp_5,pf_max_opp_5,pts_max_opp_5,+/-_max_opp_5,ts%_max_opp_5,efg%_max_opp_5,3par_max_opp_5,ftr_max_opp_5,orb%_max_opp_5,drb%_max_opp_5,trb%_max_opp_5,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,won_5,season_5
109,0.0,0.650,0.462963,0.636585,0.518519,0.547170,0.548600,0.230769,0.222222,0.793353,0.36,0.400,0.391304,0.452381,0.772727,0.578947,0.517241,0.518519,0.623529,0.635535,0.657895,0.555556,0.191419,0.464819,0.394456,0.436111,0.4496,0.757009,0.626712,0.550420,0.0,0.533969,0.318236,0.631579,0.407407,1.000000,0.545455,0.368421,0.600000,0.181818,0.16,1.0,0.230769,0.166667,0.230769,0.090909,0.4,0.4,0.333333,0.50,...,0.540081,0.440625,0.345545,0.446482,0.510448,0.512222,0.43648,0.346729,0.347945,0.576471,0.0,0.467223,0.452443,0.410526,0.392593,0.676259,0.254545,0.294737,0.800000,0.190909,0.176,1.0000,0.169231,0.300000,0.284615,0.272727,0.26,0.30,0.533333,0.60,0.337255,0.469444,0.5064,0.4828,0.76000,0.053233,0.134089,0.191074,0.144334,0.278333,0.0504,0.0902,0.489937,0.147882,0.447423,0.472727,0.482353,0.4,0.6,2021.0
111,0.0,0.775,0.629630,0.641463,0.629630,0.584906,0.635914,0.384615,0.422222,0.686416,0.32,0.375,0.347826,0.523810,0.272727,0.157895,0.241379,0.629630,0.847059,0.658314,0.676113,0.522569,0.316832,0.375267,0.287846,0.294444,0.4576,0.261682,0.195205,0.243697,0.0,0.724672,0.680572,0.421053,0.629630,1.000000,0.454545,0.421053,1.000000,0.363636,0.28,1.0,0.307692,0.388889,0.346154,0.318182,0.2,0.1,0.333333,0.75,...,0.357490,0.409375,0.350825,0.596588,0.573134,0.559444,0.58848,0.311215,0.273973,0.471429,0.0,0.367580,0.487247,0.378947,0.407407,0.632374,0.236364,0.315789,0.777867,0.254545,0.296,1.0000,0.276923,0.322222,0.261538,0.254545,0.20,0.18,0.333333,0.75,0.305882,0.369444,0.4576,0.4626,0.84000,0.068436,0.155098,0.269240,0.175138,0.410952,0.0488,0.0784,0.509015,0.174069,0.419588,0.490909,0.451765,0.4,0.6,2021.0
115,0.0,0.600,0.537037,0.531707,0.592593,0.528302,0.654036,0.333333,0.400000,0.605491,0.40,0.500,0.500000,0.523810,0.590909,0.315789,0.310345,0.666667,0.647059,0.551253,0.582996,0.505208,0.320132,0.456290,0.782516,0.594444,0.5888,0.598131,0.541096,0.323529,0.0,0.606675,0.336114,0.315789,0.259259,1.000000,0.272727,0.315789,1.000000,0.272727,0.24,1.0,0.076923,0.388889,0.230769,0.227273,0.3,0.3,0.222222,0.50,...,0.448988,0.574653,0.502970,0.416205,0.591898,0.506111,0.48608,0.321495,0.197260,0.486555,0.0,0.446961,0.472944,0.284211,0.303704,0.766187,0.309091,0.357895,0.511200,0.345455,0.320,0.9428,0.215385,0.255556,0.230769,0.263636,0.18,0.14,0.288889,0.50,0.243137,0.425000,0.4436,0.4400,0.90400,0.108234,0.185170,0.375875,0.223982,0.425476,0.0486,0.0954,0.656604,0.351220,0.312371,0.485714,0.428235,0.4,0.8,2021.0
116,0.0,0.425,0.666667,0.280488,0.518519,0.584906,0.515651,0.230769,0.200000,0.888728,0.60,0.375,0.500000,0.428571,0.636364,0.315789,0.413793,0.666667,0.411765,0.300683,0.344130,0.508681,0.146865,0.545842,0.522388,0.430556,0.6080,0.635514,0.479452,0.411765,0.0,0.348033,0.474374,0.368421,0.296296,0.401079,0.272727,0.210526,0.556000,0.136364,0.08,1.0,0.384615,0.444444,0.346154,0.363636,0.3,0.1,0.333333,0.75,...,0.487854,0.544444,0.353795,0.348401,0.448188,0.371667,0.44000,0.291589,0.354795,0.514286,0.0,0.418594,0.538498,0.357895,0.348148,0.604317,0.290909,0.294737,0.730133,0.218182,0.200,1.0000,0.138462,0.311111

In [94]:
df.loc[:, ['target', 'date', 'season', 'team', 'team_opp']].tail(20)

,target,date,season,team,team_opp
5170,1,4/19/2023,2023,MIA,MIL
5171,0,4/19/2023,2023,MIL,MIA
5172,1,4/20/2023,2023,PHI,BRK
5173,0,4/20/2023,2023,BRK,PHI
5174,2,4/20/2023,2023,SAC,GSW
5175,2,4/20/2023,2023,GSW,SAC
5176,1,4/20/2023,2023,PHO,LAC
5177,0,4/20/2023,2023,LAC,PHO
5178,2,4/21/2023,2023,BOS,ATL
5179,2,4/21/2023,2023,ATL,BOS


Điều tiếp theo chúng ta có thể làm để tăng độ chính xác của model là cho thuật toán của chúng ta biết về thông tin mà chúng ta biết ở thực tế. Những gì mà chúng ta biết ví dụ như trận đó đối thủ là ai, chơi ở sân nhà hay khách.

Tương tự như tạo target, chúng ta sẽ tạo thêm các cột mới (liên quan đến trận đấu tiếp theo), ở đó chúng ta se lấy dữ liệu từ các cột hiện tại và lùi xuống 1 hàng

In [95]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

# Ở đây, các cột mới bao gồm home_next, team_opp_next, date_next
df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

In [96]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,...,orb%_opp_5,drb%_opp_5,trb%_opp_5,ast%_opp_5,stl%_opp_5,blk%_opp_5,tov%_opp_5,usg%_opp_5,ortg_opp_5,drtg_opp_5,fg_max_opp_5,fga_max_opp_5,fg%_max_opp_5,3p_max_opp_5,3pa_max_opp_5,3p%_max_opp_5,ft_max_opp_5,fta_max_opp_5,ft%_max_opp_5,orb_max_opp_5,drb_max_opp_5,trb_max_opp_5,ast_max_opp_5,stl_max_opp_5,blk_max_opp_5,tov_max_opp_5,pf_max_opp_5,pts_max_opp_5,+/-_max_opp_5,ts%_max_opp_5,efg%_max_opp_5,3par_max_opp_5,ftr_max_opp_5,orb%_max_opp_5,drb%_max_opp_5,trb%_max_opp_5,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,won_5,season_5,home_next,team_opp_next,date_next
109,0.0,0.650,0.462963,0.636585,0.518519,0.547170,0.548600,0.230769,0.222222,0.793353,0.36,0.400,0.391304,0.452381,0.772727,0.578947,0.517241,0.518519,0.623529,0.635535,0.657895,0.555556,0.191419,0.464819,0.394456,0.436111,0.4496,0.757009,0.626712,0.550420,0.0,0.533969,0.318236,0.631579,0.407407,1.000000,0.545455,0.368421,0.600000,0.181818,0.16,1.0,0.230769,0.166667,0.230769,0.090909,0.4,0.4,0.333333,0.50,...,0.446482,0.510448,0.512222,0.43648,0.346729,0.347945,0.576471,0.0,0.467223,0.452443,0.410526,0.392593,0.676259,0.254545,0.294737,0.800000,0.190909,0.176,1.0000,0.169231,0.300000,0.284615,0.272727,0.26,0.30,0.533333,0.60,0.337255,0.469444,0.5064,0.4828,0.76000,0.053233,0.134089,0.191074,0.144334,0.278333,0.0504,0.0902,0.489937,0.147882,0.447423,0.472727,0.482353,0.4,0.6,2021.0,0.0,DET,1/1/2021
111,0.0,0.775,0.629630,0.641463,0.629630,0.584906,0.635914,0.384615,0.422222,0.686416,0.32,0.375,0.347826,0.523810,0.272727,0.157895,0.241379,0.629630,0.847059,0.658314,0.676113,0.522569,0.316832,0.375267,0.287846,0.294444,0.4576,0.261682,0.195205,0.243697,0.0,0.724672,0.680572,0.421053,0.629630,1.000000,0.454545,0.421053,1.000000,0.363636,0.28,1.0,0.307692,0.388889,0.346154,0.318182,0.2,0.1,0.333333,0.75,...,0.596588,0.573134,0.559444,0.58848,0.311215,0.273973,0.471429,0.0,0.367580,0.487247,0.378947,0.407407,0.632374,0.236364,0.315789,0.777867,0.254545,0.296,1.0000,0.276923,0.322222,0.261538,0.254545,0.20,0.18,0.333333,0.75,0.305882,0.369444,0.4576,0.4626,0.84000,0.068436,0.155098,0.269240,0.175138,0.410952,0.0488,0.0784,0.509015,0.174069,0.419588,0.490909,0.451765,0.4,0.6,2021.0,1.0,ATL,1/1/2021
115,0.0,0.600,0.537037,0.531707,0.592593,0.528302,0.654036,0.333333,0.400000,0.605491,0.40,0.500,0.500000,0.523810,0.590909,0.315789,0.310345,0.666667,0.647059,0.551253,0.582996,0.505208,0.320132,0.456290,0.782516,0.594444,0.5888,0.598131,0.541096,0.323529,0.0,0.606675,0.336114,0.315789,0.259259,1.000000,0.272727,0.315789,1.000000,0.272727,0.24,1.0,0.076923,0.388889,0.230769,0.227273,0.3,0.3,0.222222,0.50,...,0.416205,0.591898,0.506111,0.48608,0.321495,0.197260,0.486555,0.0,0.446961,0.472944,0.284211,0.303704,0.766187,0.309091,0.357895,0.511200,0.345455,0.320,0.9428,0.215385,0.255556,0.230769,0.263636,0.18,0.14,0.288889,0.50,0.243137,0.425000,0.4436,0.4400,0.90400,0.108234,0.185170,0.375875,0.223982,0.425476,0.0486,0.0954,0.656604,0.351220,0.312371,0.485714,0.428235,0.4,0.8,2021.0,0.0,UTA,1/1/2021
116,0.0,0.425,0.666667,0.280488,0.518519,0.584906,0.515651,0.230769,0.200000,0.888728,0.60,0.375,0.500000,0.428571,0.636364,0.315789,0.413793,0.666667,0.411765,0.300683,0.344130,0.508681,0.146865,0.545842,0.522388,0.430556,0.6080,0.635514,0.479452,0.411765,0.0,0.348033,0.474374,0.368421,0.296296,0.401079,0.272727,0.210526,0.556000,0.136364,0.08,1.0,0.384615,0.444444,0.346154,0.363636,0.3,0.1,0.333333,0.75,...,0.348401,0.448188,0.371667,0.44000,0.291589,0.354795,0.514286,0.0,0.418594,0.538498,0.357895,0.348148,0.604317,0.290909,0.294737,0.730133,0.218182,0.200,1.0000,0.138462,0.311111,0.238462,0.245455,0.18,0.20,0.244444,0.40,0.294118,0.3

In [97]:
df.loc[:, ['target', 'team', 'team_opp_next', 'date_next', 'home_next']].tail(20)

,target,team,team_opp_next,date_next,home_next
5170,1,MIA,MIL,4/22/2023,1.0
5171,0,MIL,MIA,4/22/2023,0.0
5172,1,PHI,BRK,4/22/2023,0.0
5173,0,BRK,PHI,4/22/2023,1.0
5174,2,SAC,None,None,NaN
5175,2,GSW,None,None,NaN
5176,1,PHO,LAC,4/22/2023,0.0
5177,0,LAC,PHO,4/22/2023,1.0
5178,2,BOS,None,None,NaN
5179,2,ATL,None,None,NaN


Lấy stat của đối thủ tiếp theo

In [98]:
full = df.merge(
    df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
    left_on=["team", "date_next"], 
    right_on=["team_opp_next", "date_next"])


In [99]:
full.loc[:, ['won','target', 'team_x', 'team_opp', 'team_opp_next_x', 'date_next']].tail(20)


,won,target,team_x,team_opp,team_opp_next_x,date_next
4756,True,0,LAC,PHO,PHO,4/18/2023
4757,False,1,PHO,LAC,LAC,4/18/2023
4758,False,0,BRK,PHI,PHI,4/20/2023
4759,True,1,PHI,BRK,BRK,4/20/2023
4760,False,1,GSW,SAC,SAC,4/20/2023
4761,True,0,SAC,GSW,GSW,4/20/2023
4762,False,1,ATL,BOS,BOS,4/21/2023
4763,True,0,BOS,ATL,ATL,4/21/2023
4764,False,1,NYK,CLE,CLE,4/21/2023
4765,True,0,CLE,NYK,NYK,4/21/2023


In [100]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]].tail(20)

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
4756,LAC,PHO,PHO,LAC,4/18/2023
4757,PHO,LAC,LAC,PHO,4/18/2023
4758,BRK,PHI,PHI,BRK,4/20/2023
4759,PHI,BRK,BRK,PHI,4/20/2023
4760,GSW,SAC,SAC,GSW,4/20/2023
4761,SAC,GSW,GSW,SAC,4/20/2023
4762,ATL,BOS,BOS,ATL,4/21/2023
4763,BOS,ATL,ATL,BOS,4/21/2023
4764,NYK,CLE,CLE,NYK,4/21/2023
4765,CLE,NYK,NYK,CLE,4/21/2023


In [101]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [102]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_opp_next_x',
 'date_next',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [103]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [104]:
predictors = list(selected_columns[sfs.get_support()])

In [105]:
predictors

['usg%',
 'ts%_max',
 '3p_opp',
 'usg%_opp',
 'ft%_max_opp',
 'stl_max_opp',
 '+/-_max_opp',
 'ftr_max_opp',
 'mp_5_x',
 'fg_5_x',
 'drb%_5_x',
 'usg%_5_x',
 'drtg_5_x',
 '+/-_max_5_x',
 'mp_opp_5_x',
 '3p_opp_5_x',
 'usg%_opp_5_x',
 'ast_max_opp_5_x',
 'stl_max_opp_5_x',
 'orb%_max_opp_5_x',
 'trb%_max_opp_5_x',
 'ortg_max_opp_5_x',
 'won_5_x',
 'home_next',
 'usg%_5_y',
 '+/-_max_5_y',
 'blk_opp_5_y',
 'usg%_opp_5_y',
 '3pa_max_opp_5_y',
 'ts%_max_opp_5_y']

In [106]:
predictions = backtest(full, rr, predictors)

In [107]:
predictions.tail(50)

,actual,prediction
4726,1,0
4727,1,1
4728,0,1
4729,0,1
4730,0,1
4731,0,0
4732,0,0
4733,0,1
4734,1,1
4735,1,1


In [108]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5818181818181818